# This code represents a trading strategy for 1 hour trading intervals
# The predictor is the close price of the ADA cryptocurrency and the minimum threshold is 0.1

In [1]:
################# IMPORTS AND SETUP ##############################


from binance import Client
import pandas as pd
from supporting_functionsM2 import *
from API_KEYS2 import get_keys

# Setting up binance client
KEY, SECRET = get_keys()
client = Client(KEY, SECRET)

# Defining setup variables for creating dataset
# start = "1513724400" # 20.12.2017
start = "1483225200" # 01.01.2017
periods = 101 # approx 8 years
TICKER = "ADAUSDT"
options = ["high", "low", "close", "volume"] # BASE OPTIONS
EPOCHS=5
THRESHOLD=0.1 # Threshold value for lableling, ie. BTCUSDT:close = 0.5 > 0.0 => label = 1, if BTCUSDT:close = -0.5 < 0.0 => 0
PREDICTOR=f"{TICKER}:close"
TIME = f"{TICKER}:time"

In [2]:
################# DOWNLOADING RAW DATA ##########################################

periods = periods
LIMIT = 720 # 720h = 30 days
start1 = start
end1 = next_30_days_unix_timestamp(start1)

############################################ DOWNLOADING DATA ###################################################################### 
data = pd.DataFrame(columns=["time", "open", "high", "low", "close", "volume"])

# downloading the first set of candlestick lines
klines = client.get_historical_klines(TICKER, client.KLINE_INTERVAL_1HOUR, limit=LIMIT, start_str=unix_to_datetime_string(start1, in_milliseconds=False), end_str=unix_to_datetime_string(end1, in_milliseconds=False))
    # print(klines)

# Converting data from list to pandas dataframe
new_data = pd.DataFrame(data=[row[0:6] for row in klines], columns=["time", "open", "high", "low", "close", "volume"])
data = pd.concat([data, new_data], ignore_index=True)

for i in range(periods - 1):
    # Moving the start and end interval to next day
    start1 = next_30_days_unix_timestamp(start1)
    end1 = next_30_days_unix_timestamp(start1)

    # downloading candlestick lines
    klines = client.get_historical_klines(TICKER, client.KLINE_INTERVAL_1HOUR, limit=LIMIT, start_str=unix_to_datetime_string(start1, in_milliseconds=False), end_str=unix_to_datetime_string(end1, in_milliseconds=False))
    # print(klines)

    # Converting data from list to pandas dataframe
    new_data = pd.DataFrame(data=[row[0:6] for row in klines], columns=["time", "open", "high", "low", "close", "volume"])

    # concatinating the new data with the existing data
    data = pd.concat([data, new_data], ignore_index=True)

# converting all time values from unix to readable string, not important, just for visual purposes and fact checking
data["time"] = data["time"].apply(unix_to_datetime_string) #converting time from 

raw_data = data
display(raw_data)


,time,open,high,low,close,volume
0,2018-04-17 04:00:00,0.25551000,0.28800000,0.25551000,0.26664000,8143693.23000000
1,2018-04-17 05:00:00,0.26660000,0.27798000,0.26010000,0.26200000,8317923.61000000
2,2018-04-17 06:00:00,0.26221000,0.26396000,0.24800000,0.25664000,8420095.41000000
3,2018-04-17 07:00:00,0.25662000,0.26300000,0.25489000,0.25698000,4686043.91000000
4,2018-04-17 08:00:00,0.25636000,0.25998000,0.25229000,0.25631000,3510038.13000000
...,...,...,...,...,...,...
61338,2025-04-18 18:00:00,0.62620000,0.62920000,0.62480000,0.62700000,3811821.50000000
61339,2025-04-18 19:00:00,0.62690000,0.62850000,0.62530000,0.62740000,1910694.50000000
61340,2025-04-18 20:00:00,0.62730000,0.62870000,0.62560000,0.62810000,2539948.40000000
61341,2025-04-18 21:00:00,0.62810000,0.63180000,0.62620000,0.63100000,6388871.80000000


In [3]:
########################################## PREPROCESSING DATA ####################################################################

# New dataobject for storing processed data
# processed_data = {f"{TICKER}:time": [], f"{TICKER}:open": [], f"{TICKER}:high": [], f"{TICKER}:low": [], f"{TICKER}:close": [], f"{TICKER}:volume": []}
processed_data = {f"{TICKER}:time": [], f"{TICKER}:high": [], f"{TICKER}:low": [], f"{TICKER}:close": [], f"{TICKER}:volume": []}

for i, o in enumerate(raw_data["open"]): #o == open, the open price value of the candle stick
    if i == 0: #Skipping the first hour to calculate the percent diff using this hour
        continue

    if o == 0:
        continue

    processed_data[f"{TICKER}:time"].append(raw_data["time"][i]) #time is the same
    # processed_data[f"{TICKER}:open"].append(percent_difference(float(data["open"][i-1]), float(o))) # percent difference between the opening price of the prior candlestick vs. open of current candle
    processed_data[f"{TICKER}:high"].append(percent_difference(float(o), float(raw_data["high"][i]))) # percent diff between open and high
    processed_data[f"{TICKER}:low"].append(percent_difference(float(o), float(raw_data["low"][i]))) # percent diff between open and low
    processed_data[f"{TICKER}:close"].append(percent_difference(float(o), float(raw_data["close"][i]))) # percent diff between open and close
    processed_data[f"{TICKER}:volume"].append(percent_difference(float(raw_data["volume"][i-1]), float(raw_data["volume"][i]))) # percent difference between the colume of the prior candlestick vs. open of current candle


# processed_data = pd.DataFrame(data=processed_data, columns=[f"{TICKER}:time", f"{TICKER}:open", f"{TICKER}:high", f"{TICKER}:low", f"{TICKER}:close", f"{TICKER}:volume"])
processed_data = pd.DataFrame(data=processed_data, columns=[f"{TICKER}:time", f"{TICKER}:high", f"{TICKER}:low", f"{TICKER}:close", f"{TICKER}:volume"])
display(processed_data)

,ADAUSDT:time,ADAUSDT:high,ADAUSDT:low,ADAUSDT:close,ADAUSDT:volume
0,2018-04-17 05:00:00,4.268567,-2.438110,-1.725431,2.139452
1,2018-04-17 06:00:00,0.667404,-5.419320,-2.124252,1.228333
2,2018-04-17 07:00:00,2.486166,-0.674149,0.140285,-44.346902
3,2018-04-17 08:00:00,1.412077,-1.587611,-0.019504,-25.095919
4,2018-04-17 09:00:00,1.361632,-1.291405,-0.370645,-23.658621
...,...,...,...,...,...
61337,2025-04-18 18:00:00,0.479080,-0.223571,0.127755,12.376273
61338,2025-04-18 19:00:00,0.255224,-0.255224,0.079758,-49.874502
61339,2025-04-18 20:00:00,0.223179,-0.271003,0.127531,32.933255
61340,2025-04-18 21:00:00,0.589078,-0.302500,0.461710,151.535496


In [4]:
############################# STATISTICAL ANALYSIS OF PROCESSED DATA #######################################
import statistics as st

close_data_processed = processed_data[f"{TICKER}:close"].to_list()
close_data_processed = close_data_processed[:-10000]
print(f"TOTAL DATAPOINTS: {len(close_data_processed)}")
print(f"MEAN CLOSE: {st.mean(close_data_processed)}")
print(f"MEDIAN CLOSE: {st.median(close_data_processed)}")

print(f"MAX CLOSE IN DATASET: {max(close_data_processed)}")
print(f"OCCURRENCES OVER 0.0: {sum(x > 0 for x in close_data_processed)} - PERCENT OF DATASET: {round(sum(x > 0 for x in close_data_processed) / len(close_data_processed), 2) * 100}%")
print(f"OCCURRENCES OVER 0.1: {sum(x > 0.1 for x in close_data_processed)} - PERCENT OF DATASET: {round(sum(x > 0.1 for x in close_data_processed) / len(close_data_processed), 2) * 100}%")
print(f"OCCURRENCES OVER 0.2: {sum(x > 0.2 for x in close_data_processed)} - PERCENT OF DATASET: {round(sum(x > 0.2 for x in close_data_processed) / len(close_data_processed), 2) * 100}%")
print(f"OCCURRENCES OVER 0.3: {sum(x > 0.3 for x in close_data_processed)} - PERCENT OF DATASET: {round(sum(x > 0.3 for x in close_data_processed) / len(close_data_processed), 2) * 100}%")
print(f"OCCURRENCES OVER 0.4: {sum(x > 0.4 for x in close_data_processed)} - PERCENT OF DATASET: {round(sum(x > 0.4 for x in close_data_processed) / len(close_data_processed), 2) * 100}%")
print(f"OCCURRENCES OVER 0.5: {sum(x > 0.5 for x in close_data_processed)} - PERCENT OF DATASET: {round(sum(x > 0.5 for x in close_data_processed) / len(close_data_processed), 2) * 100}%")
print("----------------------------------------------------------------------")

print("EVALUATION DATA")
close_data_processed = processed_data[f"{TICKER}:close"].to_list()
close_data_processed = close_data_processed[-10000:]
print(f"TOTAL DATAPOINTS: {len(close_data_processed)}")
print(f"MEAN CLOSE: {st.mean(close_data_processed)}")
print(f"MEDIAN CLOSE: {st.median(close_data_processed)}")

print(f"MAX CLOSE IN DATASET: {max(close_data_processed)}")
print(f"OCCURRENCES OVER 0.0: {sum(x > 0 for x in close_data_processed)} - PERCENT OF DATASET: {round(sum(x > 0 for x in close_data_processed) / len(close_data_processed), 2) * 100}%")
print(f"OCCURRENCES OVER 0.1: {sum(x > 0.1 for x in close_data_processed)} - PERCENT OF DATASET: {round(sum(x > 0.1 for x in close_data_processed) / len(close_data_processed), 2) * 100}%")
print(f"OCCURRENCES OVER 0.2: {sum(x > 0.2 for x in close_data_processed)} - PERCENT OF DATASET: {round(sum(x > 0.2 for x in close_data_processed) / len(close_data_processed), 2) * 100}%")
print(f"OCCURRENCES OVER 0.3: {sum(x > 0.3 for x in close_data_processed)} - PERCENT OF DATASET: {round(sum(x > 0.3 for x in close_data_processed) / len(close_data_processed), 2) * 100}%")
print(f"OCCURRENCES OVER 0.4: {sum(x > 0.4 for x in close_data_processed)} - PERCENT OF DATASET: {round(sum(x > 0.4 for x in close_data_processed) / len(close_data_processed), 2) * 100}%")
print(f"OCCURRENCES OVER 0.5: {sum(x > 0.5 for x in close_data_processed)} - PERCENT OF DATASET: {round(sum(x > 0.5 for x in close_data_processed) / len(close_data_processed), 2) * 100}%")
print("----------------------------------------------------------------------")

TOTAL DATAPOINTS: 51342
MEAN CLOSE: 0.006801309768745919
MEDIAN CLOSE: 0.0
MAX CLOSE IN DATASET: 15.037173235270618
OCCURRENCES OVER 0.0: 25119 - PERCENT OF DATASET: 49.0%
OCCURRENCES OVER 0.1: 22509 - PERCENT OF DATASET: 44.0%
OCCURRENCES OVER 0.2: 19542 - PERCENT OF DATASET: 38.0%
OCCURRENCES OVER 0.3: 16858 - PERCENT OF DATASET: 33.0%
OCCURRENCES OVER 0.4: 14407 - PERCENT OF DATASET: 28.000000000000004%
OCCURRENCES OVER 0.5: 12369 - PERCENT OF DATASET: 24.0%
----------------------------------------------------------------------
EVALUATION DATA
TOTAL DATAPOINTS: 10000
MEAN CLOSE: 0.006578835555302232
MEDIAN CLOSE: 0.0
MAX CLOSE IN DATASET: 33.70907967881409
OCCURRENCES OVER 0.0: 4889 - PERCENT OF DATASET: 49.0%
OCCURRENCES OVER 0.1: 4324 - PERCENT OF DATASET: 43.0%
OCCURRENCES OVER 0.2: 3791 - PERCENT OF DATASET: 38.0%
OCCURRENCES OVER 0.3: 3265 - PERCENT OF DATASET: 33.0%
OCCURRENCES OVER 0.4: 2789 - PERCENT OF DATASET: 28.000000000000004%
OCCURRENCES OVER 0.5: 2383 - PERCENT OF DAT

In [5]:
########################### LABELING THE DATA ##################################


# column_labels = ["BTCUSDT:time"] # name of the columns for the return dataframe
column_labels = ["time"] # name of the columns for the return dataframe

# filling up the list with labels for the columns
for roundd in range(EPOCHS):
    for option in options:
        column_labels.append(f"{TICKER}:{option}{roundd}")

column_labels.append("Label")


# filling up list of data, row by row in the dataset
labelled_data_rows = [] # this list stores all the rows filled with all the data
for i in range(len(processed_data[TIME]) - EPOCHS): #looping from the third element to the third last element, with stepsize 1, if epoch=3
    data_row = []

    data_row.append(processed_data[TIME][i + EPOCHS - 1])

    for t in range(EPOCHS):
        for option in options:
            data_row.append(processed_data[f"{TICKER}:{option}"][i + t])

    if processed_data[PREDICTOR][i + EPOCHS] >= THRESHOLD: # here we use the threshold, AND APPEND THE LABEL
        data_row.append(1)
    else:
        data_row.append(0)

    labelled_data_rows.append(data_row)


labelled_data_frame = pd.DataFrame(labelled_data_rows, columns=column_labels)
display(labelled_data_frame)

,time,ADAUSDT:high0,ADAUSDT:low0,ADAUSDT:close0,ADAUSDT:volume0,ADAUSDT:high1,ADAUSDT:low1,ADAUSDT:close1,ADAUSDT:volume1,ADAUSDT:high2,...,ADAUSDT:volume2,ADAUSDT:high3,ADAUSDT:low3,ADAUSDT:close3,ADAUSDT:volume3,ADAUSDT:high4,ADAUSDT:low4,ADAUSDT:close4,ADAUSDT:volume4,Label
0,2018-04-17 09:00:00,4.268567,-2.438110,-1.725431,2.139452,0.667404,-5.419320,-2.124252,1.228333,2.486166,...,-44.346902,1.412077,-1.587611,-0.019504,-25.095919,1.361632,-1.291405,-0.370645,-23.658621,0
1,2018-04-17 10:00:00,0.667404,-5.419320,-2.124252,1.228333,2.486166,-0.674149,0.140285,-44.346902,1.412077,...,-25.095919,1.361632,-1.291405,-0.370645,-23.658621,0.288780,-1.190244,0.050732,-20.581889,1
2,2018-04-17 11:00:00,2.486166,-0.674149,0.140285,-44.346902,1.412077,-1.587611,-0.019504,-25.095919,1.361632,...,-23.658621,0.288780,-1.190244,0.050732,-20.581889,2.192059,-0.538264,1.915126,-6.651501,0
3,2018-04-17 12:00:00,1.412077,-1.587611,-0.019504,-25.095919,1.361632,-1.291405,-0.370645,-23.658621,0.288780,...,-20.581889,2.192059,-0.538264,1.915126,-6.651501,2.157336,-1.919761,-1.218531,95.086660,0
4,2018-04-17 13:00:00,1.361632,-1.291405,-0.370645,-23.658621,0.288780,-1.190244,0.050732,-20.581889,2.192059,...,-6.651501,2.157336,-1.919761,-1.218531,95.086660,0.849529,-0.306451,-0.108616,-51.258434,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61332,2025-04-18 17:00:00,0.226868,-0.340301,0.145843,-26.006756,0.420780,-0.388412,-0.097103,79.098599,0.113397,...,47.491798,0.600454,-0.097371,0.600454,-20.884357,1.080819,-0.032263,1.000161,37.673761,1
61333,2025-04-18 18:00:00,0.420780,-0.388412,-0.097103,79.098599,0.113397,-0.631784,-0.178195,47.491798,0.600454,...,-20.884357,1.080819,-0.032263,1.000161,37.673761,0.479080,-0.223571,0.127755,12.376273,0
61334,2025-04-18 19:00:00,0.113397,-0.631784,-0.178195,47.491798,0.600454,-0.097371,0.600454,-20.884357,1.080819,...,37.673761,0.479080,-0.223571,0.127755,12.376273,0.255224,-0.255224,0.079758,-49.874502,1
61335,2025-04-18 20:00:00,0.600454,-0.097371,0.600454,-20.884357,1.080819,-0.032263,1.000161,37.673761,0.479080,...,12.376273,0.255224,-0.255224,0.079758,-49.874502,0.223179,-0.271003,0.127531,32.933255,1


# Model Training

This code block divides the dataset into a training dataset and trains a new model using the **AutoGluon Tabular** predictor.

In [6]:
################################ TRAINING NEW MODEL #######################################
from autogluon.tabular import TabularDataset, TabularPredictor

# defining training size and colums to use for training within the labelled dataset
VALIDATION_SIZE = 10000
columns_to_use = [f"{TICKER}:high0", f"{TICKER}:low0", f"{TICKER}:close0", f"{TICKER}:volume0", f"{TICKER}:high1", f"{TICKER}:low1", f"{TICKER}:close1", f"{TICKER}:volume1", f"{TICKER}:high2", f"{TICKER}:low2", f"{TICKER}:close2", f"{TICKER}:volume2", f"{TICKER}:high3", f"{TICKER}:low3", f"{TICKER}:close3", f"{TICKER}:volume3", f"{TICKER}:high4", f"{TICKER}:low4", f"{TICKER}:close4", f"{TICKER}:volume4", "Label"]
LABEL = "Label"

# defining training data
training_dataframe = labelled_data_frame.iloc[:-VALIDATION_SIZE].copy()
train_data_frame2 = training_dataframe[columns_to_use]
train_tabular_dataset = TabularDataset(train_data_frame2)

# # Training model -> TabularPredictor
# predictor = TabularPredictor(label=label, eval_metric="balanced_accuracy", positive_class=1).fit(train_tabular_dataset, num_bag_folds=5, num_bag_sets=5, num_stack_levels=3)
# predictor = TabularPredictor(label=label, eval_metric="accuracy").fit(train_tabular_dataset, presets="high_quality")
predictor = TabularPredictor(label=LABEL).fit(train_tabular_dataset)



c:\Users\47981\Desktop\MasterThesis\Repo\MasterThesis\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No path specified. Models will be saved in: "AutogluonModels\ag-20250603_104644"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          12
Memory Avail:       3.23 GB / 15.92 GB (20.3%)
Disk Space Avail:   113.35 GB / 475.69 GB (23.8%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Defau

# Model Evaluation

This code block divides the dataset into a validation dataset and evaluates the model using the **AutoGluons** inbuilt evaluation library. In addition the model is backtested using the validation set to measure its performance and calulate its "expected return" over the period.

In [7]:
#################### MODEL EVALUATION ################################

# Defining the testing set using the training size and columns to use
testing_dataframe = labelled_data_frame.tail(VALIDATION_SIZE).copy()
#display(testing_dataframe)
test_data_frame2 = testing_dataframe[columns_to_use]
test_tabular_dataset = TabularDataset(test_data_frame2)

######## Making predictions
y_pred = predictor.predict(test_tabular_dataset.drop(columns=[LABEL]))
display(y_pred)


#### Evaluation
eval_report = predictor.evaluate(test_tabular_dataset, detailed_report=True)
display(eval_report)

feature_importance = predictor.feature_importance(test_tabular_dataset)
display(feature_importance)


51337    0
51338    0
51339    0
51340    0
51341    0
        ..
61332    0
61333    0
61334    0
61335    0
61336    0
Name: Label, Length: 10000, dtype: int64

{'accuracy': 0.5683,
 'balanced_accuracy': 0.5190982985633719,
 'mcc': 0.05563701144249248,
 'roc_auc': 0.5497355786312232,
 'f1': 0.2371443717971373,
 'precision': 0.5026217228464419,
 'recall': 0.15518038852913968,
 'confusion_matrix':       0    1
 0  5012  664
 1  3653  671,
 'classification_report': {'0': {'precision': 0.5784189267166763,
   'recall': 0.883016208597604,
   'f1-score': 0.6989749668781814,
   'support': 5676.0},
  '1': {'precision': 0.5026217228464419,
   'recall': 0.15518038852913968,
   'f1-score': 0.2371443717971373,
   'support': 4324.0},
  'accuracy': 0.5683,
  'macro avg': {'precision': 0.5405203247815591,
   'recall': 0.5190982985633719,
   'f1-score': 0.46805966933765936,
   'support': 10000.0},
  'weighted avg': {'precision': 0.5456442157631869,
   'recall': 0.5683,
   'f1-score': 0.49927941756513794,
   'support': 10000.0}}}

Computing feature importance via permutation shuffling for 20 features using 5000 rows with 5 shuffle sets...
	73.61s	= Expected runtime (14.72s per shuffle set)
	18.51s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
ADAUSDT:close4,0.02484,0.007129,0.000732,5,0.039519,0.010161
ADAUSDT:low2,0.00052,0.001262,0.204459,5,0.003118,-0.002078
ADAUSDT:volume4,0.00024,0.001090,0.324131,5,0.002484,-0.002004
ADAUSDT:low3,-0.00008,0.001814,0.536898,5,0.003656,-0.003816
ADAUSDT:close3,-0.00008,0.009210,0.507283,5,0.018884,-0.019044
ADAUSDT:low1,-0.00016,0.001090,0.620404,5,0.002084,-0.002404
ADAUSDT:volume0,-0.00028,0.001092,0.701484,5,0.001968,-0.002528
ADAUSDT:volume3,-0.00028,0.002390,0.596858,5,0.004641,-0.005201
ADAUSDT:high2,-0.00060,0.002005,0.729980,5,0.003528,-0.004728
ADAUSDT:volume2,-0.00060,0.001049,0.865004,5,0.001560,-0.002760


In [8]:
import re
import pandas as pd

METRIC_RE = re.compile(r":([a-zA-Z]+?)(\d+)$")   # capture metric & hour suffix


def analyze_feature_importance(feature_names, importances):
    """
    Print a short report on a model’s feature-importance vector.

    Parameters
    ----------
    feature_names : list[str]
        All feature names, e.g. ["SOLUSDT:high4", "SOLUSDT:low2", ...]
    importances : list[float]
        Matching importance values, same length/order as `feature_names`
    """
    if len(feature_names) != len(importances):
        raise ValueError("feature_names and importances must be the same length!")

    # ── Build a DataFrame ────────────────────────────────────────────────────
    df = pd.DataFrame(
        {"feature": feature_names, "importance": importances}
    ).assign(
        metric=lambda d: d["feature"].str.extract(METRIC_RE)[0].str.lower(),
        hour=lambda d: pd.to_numeric(
            d["feature"].str.extract(METRIC_RE)[1], errors="coerce"
        ),
    )

    # Drop rows we could not parse
    df = df.dropna(subset=["metric", "hour"]).copy()
    df["hour"] = df["hour"].astype(int)

    # ── 1. % of positive importances ────────────────────────────────────────
    pct_pos = (df["importance"] > 0).mean() * 100

    # ── 2. Rank by hour (sum of importances per lag) ────────────────────────
    hour_scores = df.groupby("hour")["importance"].sum().sort_values(ascending=False)
    hour_ranking = hour_scores.index.tolist()

    # ── 3. Rank by metric (sum of importances per metric) ───────────────────
    metric_scores = (
        df.groupby("metric")["importance"].sum().sort_values(ascending=False)
    )
    metric_ranking = metric_scores.index.tolist()

    # ── Report ──────────────────────────────────────────────────────────────
    print(f"{pct_pos:.1f}% of the {len(df)} features have **positive** importance.\n")

    print("Ranking by HOUR (most → least important):")
    for i, h in enumerate(hour_ranking, 1):
        print(f"  {i}. h{h:<1}   sum importance = {hour_scores[h]:.6f}")

    print("\nRanking by METRIC (most → least important):")
    for i, m in enumerate(metric_ranking, 1):
        print(f"  {i}. {m:<6} sum importance = {metric_scores[m]:.6f}")

    top_features = (
        df.loc[df["importance"].abs().sort_values(ascending=False).index]
        .head(5)[["feature", "importance"]]
    )
    print("\nTop 5 individual features:")
    for feat, imp in top_features.itertuples(index=False):
        print(f"  {feat:<25} {imp:.6f}")




analyze_feature_importance(feature_importance.index.to_list(), feature_importance['importance'].to_list())

15.0% of the 20 features have **positive** importance.

Ranking by HOUR (most → least important):
  1. h4   sum importance = 0.020920
  2. h2   sum importance = -0.002000
  3. h3   sum importance = -0.002600
  4. h1   sum importance = -0.004240
  5. h0   sum importance = -0.004880

Ranking by METRIC (most → least important):
  1. close  sum importance = 0.021040
  2. volume sum importance = -0.002400
  3. low    sum importance = -0.004000
  4. high   sum importance = -0.007440

Top 5 individual features:
  ADAUSDT:close4            0.024840
  ADAUSDT:low4              -0.002800
  ADAUSDT:high3             -0.002160
  ADAUSDT:high0             -0.001840
  ADAUSDT:high1             -0.001480


In [9]:
# ANALYSIS OF PREDICTIONS AND PROBABILITIES

import statistics as st

# making and processing probabilities from evaluation dataset
y_prob = predictor.predict_proba(test_tabular_dataset.drop(columns=[LABEL]))

validation_probabilities = pd.DataFrame(y_prob).reset_index(drop=True) # probability for each prediction
validation_predictions = pd.DataFrame(y_pred).reset_index(drop=True)
validation_dataset = pd.DataFrame(test_data_frame2).reset_index(drop=True)
#display(validation_probabilities)
#display(validation_predictions)
#display(validation_dataset)

print(f"MEAN PROB 1 CLASSIFICATION: {st.mean(validation_probabilities[1].to_list())}")
print(f"MAX PROB 1 CLASSIFICATION: {max(validation_probabilities[1].to_list())}")
print(f"MIN PROB 1 CLASSIFICATION: {min(validation_probabilities[1].to_list())}")

count_above_07 = sum(1 for num in validation_probabilities[1].to_list() if num > 0.5)
print("COUNT OF NUMBERS > 0.5:", count_above_07)

count_above_08 = sum(1 for num in validation_probabilities[1].to_list() if num > 0.55)
print("COUNT OF NUMBERS > 0.55:", count_above_08)

count_above_09 = sum(1 for num in validation_probabilities[1].to_list() if num > 0.6)
print("COUNT OF NUMBERS > 0.6:", count_above_09)








MEAN PROB 1 CLASSIFICATION: 0.42546091130673885
MAX PROB 1 CLASSIFICATION: 0.8602320551872253
MIN PROB 1 CLASSIFICATION: 0.25118935108184814
COUNT OF NUMBERS > 0.5: 1335
COUNT OF NUMBERS > 0.55: 508
COUNT OF NUMBERS > 0.6: 185


In [ ]:
import matplotlib.pyplot as plt

# --- Constants ---
MINIMUM_PROBABILITY = 0.7
LEVERAGE = 1
GAIN = 0.5  # Interpreted as a percentage (e.g., 0.5% if multiplied by /100)

# --- Tracking variables ---
correct_trades = 0
total_trades = 0
bad_trades = []
all_trades = []

initial_investment = 100
current_investment = initial_investment
investment_history = []

# Go through probabilities alongside their index
for idx, prob in enumerate(validation_probabilities[1].to_list()):
    
    # Check if the predicted probability meets the threshold
    if prob >= MINIMUM_PROBABILITY:
        total_trades += 1
        
        # Check if the prediction was correct
        if validation_predictions["Label"][idx] == validation_dataset["Label"][idx]:
            correct_trades += 1
            
            # Record the "gain" in your trade list
            all_trades.append(GAIN * LEVERAGE)
            
            # Update current_investment by a factor of (1 + gain%)
            current_investment *= 1 + (GAIN / 100 * LEVERAGE)
        
        else:
            # A "bad" (wrong) trade - since this focuses on shorting the true close is multiplied by -1
            # Grab the next close price; watch out for index out-of-range in real code
            true_close = (validation_dataset[f"{TICKER}:close4"][idx + 1])
            
            # Record the trade details
            bad_trades.append(round(true_close * LEVERAGE, 3))
            all_trades.append(true_close * LEVERAGE)

            # Update current_investment by (1 + some factor of true_close?)
            current_investment *= 1 + (true_close / 100 * LEVERAGE)
    
    else:
        all_trades.append(0)

    # In all cases, record the current investment amount
    investment_history.append(current_investment)

# --- After the loop, calculate stats ---
wrong_trades = total_trades - correct_trades
win_rate = (correct_trades / total_trades * 100) if total_trades else 0
total_return = current_investment - initial_investment

# --- Print results ---
print(f"CORRECT: {correct_trades}")
print(f"WRONG: {wrong_trades}")
print(f"NUMBER OF TRADES: {total_trades}")
print(f"WIN RATE: {round(win_rate, 2)}%")
print(f"RETURN: {round(total_return, 2)}")
print(f"INVESTMENT VALUE: {round(current_investment, 2)}")
print(f"SHARP RATIO: {calculate_sharpe_ratio(all_trades)}")
sharpe_ratio9999 = calculate_sharpe_ratio(validation_dataset[f"{TICKER}:close4"].to_list())
print(f"SHARP RATIO ONLY HOLDING ASSET: {sharpe_ratio9999}")
print(f"MEAN RETURN BAD TRADES: {st.mean(bad_trades)}")
print("-" * 34)

# Print information about bad trades
for trade in bad_trades:
    print(trade)

# --- Plot the investment history ---
true_closes_list = validation_dataset[f"{TICKER}:close4"].to_list()
true_close_base = 100
true_close_base_list = []

for close in true_closes_list:
    true_close_base *= 1 + (close / 100)
    true_close_base_list.append(true_close_base)


# # Create a figure with two subplots next to each other
# fig, axes = plt.subplots(ncols=2, figsize=(12, 5))  # Adjust figsize as needed

# # Plot investment history
# axes[0].plot(investment_history)
# axes[0].set_title("Investment Value Over Time")
# axes[0].set_xlabel("Time (Hours)")
# axes[0].set_ylabel("Investment Value")

# # Plot true closes
# axes[1].plot(true_close_base_list)
# axes[1].set_title("True Closes Over Time")
# axes[1].set_xlabel("Time (Hours)")
# axes[1].set_ylabel("Asset close price fluctuations")

# plt.tight_layout()
# plt.show()

plt.figure(figsize=(10,5))
plt.plot(investment_history, label="Investment History", color='red', linewidth=2.5)
plt.plot(true_close_base_list, label="True Closes", color='gray', linewidth=1)
plt.xlabel("Trade Index")
plt.ylabel("Value")
plt.title("Investment Value and True Closes Over Trades")
plt.legend()
plt.show()

In [ ]:
############### ANALYSIS OF TRADING USING CLOSING PRICE ################################

# --- Constants ---
MINIMUM_PROBABILITY = 0.8
TRADING_FEE = 0.25

# --- Tracking variables ---
correct_trades = 0
total_trades = 0
wrong_trades = 0

all_trades = []
good_trades = []
bad_trades = []

# Go through probabilities alongside their index
for idx, prob in enumerate(validation_probabilities[1].to_list()):
    
    # Check if the predicted probability meets the threshold
    if prob >= MINIMUM_PROBABILITY:
        true_close = validation_dataset[f"{TICKER}:close4"][idx + 1] - TRADING_FEE
        
        if true_close >= 0:
            correct_trades += 1
            good_trades.append(true_close)
        else:
            bad_trades.append(true_close)



        total_trades += 1
        all_trades.append(true_close)
            

# --- After the loop, calculate stats ---
wrong_trades = total_trades - correct_trades
win_rate = (correct_trades / total_trades * 100) if total_trades else 0

# --- Print results ---
print(f"CORRECT: {correct_trades}")
print(f"WRONG: {wrong_trades}")
print(f"NUMBER OF TRADES: {total_trades}")
print(f"WIN RATE: {round(win_rate, 2)}%")
print("-" * 34)

print(f"MEAN GOOD TRADES: {st.mean(good_trades)}")
print(f"MEAN BAD TRADES: {st.mean(bad_trades)}")

investtt = 100
investment_history2 = []
for trade in all_trades:
    investtt *= 1 + (trade / 100)
    investment_history2.append(investtt)
    # print(trade)

print(f"RETURN: {round(investtt - 100, 3)} %")

# --- Plot the investment history ---
plt.plot(investment_history2)
plt.xlabel("Trade Index")
plt.ylabel("Investment Value")
plt.title("Investment Value Over Trades")
plt.show()

In [ ]:
############### ANALYSIS OF TRADING USING CLOSING PRICE ################################
from matplotlib import pyplot as plt



# --- Constants ---
MINIMUM_PROBABILITY = 0.5
TRADING_FEE = 0.0
LEVERAGE = 1

# --- Tracking variables ---
correct_trades = 0
total_trades = 0
wrong_trades = 0

all_trades = []
good_trades = []
bad_trades = []
investment_value = 100
investment_history = []


# Go through probabilities alongside their index
for idx, prob in enumerate(validation_probabilities[1].to_list()):
    
    # Check if the predicted probability meets the threshold
    if prob >= MINIMUM_PROBABILITY:
        true_close = (validation_dataset[f"{TICKER}:close4"][idx + 1] - TRADING_FEE) * LEVERAGE # FEE AND LEVERAGE ADJUSTED FOR
        
        if true_close >= 0:
            correct_trades += 1
            good_trades.append(true_close)
        else:
            bad_trades.append(true_close)



        total_trades += 1
        all_trades.append(true_close)

        investment_value *= 1 + (true_close / 100)
        investment_history.append(investment_value)
    
    else:
        investment_history.append(investment_value)
        all_trades.append(0)

            

# --- After the loop, calculate stats ---
wrong_trades = total_trades - correct_trades
win_rate = (correct_trades / total_trades * 100) if total_trades else 0

# --- Print results ---
#print(f"CORRECT: {correct_trades}")
#print(f"WRONG: {wrong_trades}")
print(f"NUMBER OF TRADES: {total_trades}")
#print(f"NUMBER OF TRADES 2: {len(investment_history)}")
print(f"ACCURACY: {round(win_rate, 2)}%")
print(f"RETURN: {investment_value - 100} %")
print(f"SHARPE RATIO: {calculate_sharpe_ratio(all_trades)}")
sharpe_ratio9999 = calculate_sharpe_ratio(validation_dataset[f"{TICKER}:close4"].to_list())
print(f"SHARPE RATIO ONLY HOLDING ASSET: {sharpe_ratio9999}")
print(f"MEAN win: {st.mean(good_trades)}")
print(f"MEAN loss: {st.mean(bad_trades)}")
#print(f"INVESTMENT VALUE {investment_value}")

# for trade in all_trades:
#     print(trade)

# --- Plot the investment history ---
true_closes_list = validation_dataset[f"{TICKER}:close4"].to_list()
true_close_base = 100
true_close_base_list = []

for close in true_closes_list:
    true_close_base *= 1 + (close / 100)
    true_close_base_list.append(true_close_base)


plt.figure(figsize=(10,5))
plt.plot(investment_history, label="Trading Results", color='red', linewidth=2.5)
plt.plot(true_close_base_list, label="True Closes", color='gray', linewidth=1)
plt.xlabel("Back-test period (h)")
plt.ylabel("Value")
plt.title("Investment Value and True Closes Over Trades")
plt.legend()
plt.show()